In [ ]:
import pandas as pd
import numpy as np
from sqlalchemy import create_engine

In [ ]:
usuario = 'siempreread'
contrasena = 'S13mpr3_Ch1l3'
url_servidor = '209.151.148.250'
puerto = '3306'
esquema = 'EspoCRM2'
plugin_autenticacion = 'mysql_native_password'

# Conexión a la base de datos

c_conexion = 'mysql+mysqlconnector://{0}:{1}@{2}:{3}/{4}?auth_plugin={5}'
c_conexion = c_conexion.format(usuario, contrasena, url_servidor, puerto,
                               esquema, plugin_autenticacion)
motor_mysql_mariadb = create_engine(c_conexion)

In [ ]:
#ojo modificar año de la tabla  para ver bien los datos

sql = 'select *, month(created_at) as mes, year(created_at) as ano from contact where CHAR_LENGTH(rut_asegurado) > 6 AND CHAR_LENGTH(rut) > 6'
#dataframe desde consulta SQL ######   and month(created_at) = 6
datos = pd.read_sql_query(sql, motor_mysql_mariadb)

#tabla de emails
listado_email = 'SELECT t1.id,t3.lower as email FROM contact t1 left JOIN entity_email_address t2 ON t1.id = t2.entity_id left JOIN email_address t3 ON t2.email_address_id = t3.id '
emails = pd.read_sql_query(listado_email, motor_mysql_mariadb)

#tabla excel con ruts faltantes
ruts_faltantes = 'C:/Users/jmartinez/Desktop/casos nuevos/casosfaltantes.xlsx'
listado_faltantes = pd.read_excel(ruts_faltantes,index=False)

#archivo de salida modificar segun archivo
archivo = 'C:/Users/jmartinez/Desktop/casos nuevos/clientes/1/2022.xlsx'

In [ ]:
data20 = pd.merge(left=datos,right=emails, how='left', left_on='id', right_on='id')